In [2]:
from dotenv import load_dotenv
import openai
import os
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader, 
    StorageContext,
    load_index_from_storage,
    Document,
    PromptTemplate
)
load_dotenv()

True

In [3]:
import requests
from bs4 import BeautifulSoup

def get_wikipedia_page(title, lang):
    response = requests.get(f"https://{lang}.wikipedia.org/wiki/{title}")
    soup = BeautifulSoup(response.text, 'html.parser')
    tags = soup.find_all()
    texts = []
    text = ''
    for tag in tags:
        if tag.name == "h2":
            if text:
                texts.append(text)
            text = f'Sección: {tag.text} \n'
            
        if tag.name == "h3":
            text += f'Subsección: {tag.text} \n'

        if tag.name == "p":
            text += tag.text + '\n'
    
    documents = [
        Document(
            title=title,
            text=text,
            metadata={}
        )
        for text in texts
    ]

    return documents

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
Settings.embed_model = embed_model

In [5]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename, encoding='latin1') as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [6]:
import os
PERSIST_DIR = "lyrics_store_shakira"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("340_Shakira.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [7]:
lyrics_engine = index.as_query_engine()

In [8]:
PERSIST_DIR_WIKI = "wiki_shakira"

if not os.path.exists(PERSIST_DIR_WIKI):
    documents = get_wikipedia_page("Shakira", "es")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR_WIKI)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR_WIKI) 
    index = load_index_from_storage(storage_context)

In [9]:
wiki_engine = index.as_query_engine()

In [10]:
qa_template_str = """
    You are an expert in Shakira's music, and your task is to guide and teach the user about her songs, focusing on their themes, lyrics, and cultural context. Answer the user's queries only using supported data from the tools available. One tool contains the complete context of all the songs Shakira has ever performed, and the second contains the full Wikipedia page about her. Avoid using pre-trained information; rely solely on the provided tools for your answers.

    Context information is below:
    {context_str}
    Given the context information and no prior knowledge, answer the query with the following structure. Include detailed source information and direct quotes from the lyrics where necessary. Organize your response with bullet points for clarity.

    Canción:
    Citas directas: Include relevant lyrics or specific lines.
    Temática: Explain the theme of the song.
    Sentimiento: Describe the tone or sentiment of the song.
    Artista:
    Álbum: Specify the album where the song was released.
    Año: Mention the year of release.
    Contexto general: Provide background related to Shakira's life or events during the creation of the song, considering both the lyrics and her personal or artistic context at the time.
    Query: {query_str}
    Answer:
"""
qa_template = PromptTemplate(qa_template_str)

In [11]:
lyrics_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)
wiki_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [12]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

description_lyrics = """
    Contains the complete context of all the songs Shakira has ever performed. 
    This includes full lyrics, themes, and musical details, 
    allowing you to provide accurate and detailed responses about each song based on its content.
    """

description_wiki = """
    Provides the entire Wikipedia page of Shakira, which includes her biography, 
    discography, personal life, and career milestones. 
    This tool offers background information related to her life, career, and the 
    cultural or personal context surrounding her songs.
"""
tools = [
    QueryEngineTool(
        query_engine=lyrics_engine,
        metadata=ToolMetadata(
            name="shakira_songs_lyrics",
            description=description_lyrics,
            return_direct=False
        )
    ),
    QueryEngineTool(
        query_engine=wiki_engine,
        metadata=ToolMetadata(
            name="shakira_wikipedia",
            description=description_wiki,
            return_direct=False
        )
    )
]
agent = OpenAIAgent.from_tools(tools=tools, verbose=True)

In [17]:
response = agent.chat("que canciones de sharira habla de despecho?")

Added user message to memory: que canciones de sharira habla de despecho?
=== Calling Function ===
Calling function: shakira_songs_lyrics with args: {"input":"Shakira heartbreak songs"}
Got output: - Canción: El amor perdona
- Citas directas: "El amor perdona, El amor no miente, El amor bonito nunca es suficiente"
- Temática: La canción habla sobre el perdón en el amor, la entrega sin palabras, el dolor que puede causar, y la superación de una relación tóxica.
- Sentimiento: El tono de la canción es melancólico y reflexivo, con una sensación de liberación y empoderamiento al final.
- Artista: Shakira
- Álbum: Desconocido
- Año: Desconocido
- Contexto general: En esta canción, Shakira explora la idea de perdonar en el amor, pero también reconoce que el amor verdadero no debe basarse en mentiras ni en sufrimiento. La letra refleja una experiencia personal de desilusión y dolor en una relación, pero también muestra el proceso de sanación y recuperación emocional.

Es importante tener en c

In [20]:
agent.chat("que canciones de shakira son las mas populares del 2022?")

Added user message to memory: que canciones de shakira son las mas populares del 2022?
=== Calling Function ===
Calling function: shakira_songs_lyrics with args: {"input":"Shakira popular songs 2022"}
Got output: Canción: Brujeria en español

Citas directas: 
- "What kind of witchcraft dominates my heart, A Halloween lies, lies"
- "But I love plants and would end I, It's like a coven by hiding in my fantasies"

Temática: The song "Brujeria en español" explores themes of love, attraction, and the mysterious or magical elements that can be associated with romantic feelings. The lyrics depict a sense of being under a spell or enchanted by someone, using witchcraft as a metaphor for the intense emotions and desires experienced in a relationship.

Sentimiento: The tone of the song is passionate and somewhat mystical, with a mix of fascination and confusion about the powerful emotions the singer is experiencing. There is a sense of being entranced or bewitched by love, leading to a blend of 

AgentChatResponse(response='Algunas de las canciones más populares de Shakira en el año 2022 incluyen:\n\n- "Hips Don\'t Lie"\n- "Whenever, Wherever"\n- "Waka Waka (This Time for Africa)"\n- "La Tortura"\n- "Chantaje"\n\nEstas canciones han sido ampliamente exitosas y han contribuido al reconocimiento global de Shakira como una artista talentosa y versátil. Si deseas más información sobre alguna de estas canciones o tienes alguna otra pregunta, no dudes en decírmelo.', sources=[ToolOutput(content='Canción: Brujeria en español\n\nCitas directas: \n- "What kind of witchcraft dominates my heart, A Halloween lies, lies"\n- "But I love plants and would end I, It\'s like a coven by hiding in my fantasies"\n\nTemática: The song "Brujeria en español" explores themes of love, attraction, and the mysterious or magical elements that can be associated with romantic feelings. The lyrics depict a sense of being under a spell or enchanted by someone, using witchcraft as a metaphor for the intense emo

In [23]:
agent.chat("cual seria tu canción favorita de shakira si tuvieras preferencias personales?")

Added user message to memory: cual seria tu canción favorita de shakira si tuvieras preferencias personales?


AgentChatResponse(response='¡Como asistente de inteligencia artificial, no tengo preferencias personales ni emociones! Sin embargo, puedo proporcionarte información sobre las canciones de Shakira y ayudarte a descubrir cuál podría ser tu favorita. ¿Te gustaría saber más sobre alguna canción en particular o necesitas ayuda con algo más?', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)